In [2]:
import random

In [3]:
def pairings(fighters):
    all_keys = list(fighters.keys())
    if len(all_keys)%2 != 0:
        raise ValueError('Must have even number of fighters')
    fights_num = int(len(all_keys)/2)
    random.shuffle(all_keys)
    all_pairs = []
    for i in range(fights_num):
        all_pairs.append((all_keys[2*i], all_keys[2*i+1]))
    return all_pairs

In [311]:
fighters = {}
lost_fighters = []
NUM_F = 40
START_HP = 12

for i in range(NUM_F):
    fighters[i] = START_HP

In [4]:
def clean(fighters, lost_keys):
    fighters_to_remove = []
    for f in fighters.keys():
        if fighters[f] <=0:
            fighters_to_remove.append(f)
    for f in fighters_to_remove:
        fighters.pop(f)
        lost_keys.append(f)
    return fighters, lost_keys

def conduct_round(fighters, lost_keys, v):
    if len(fighters.keys())%2 !=0:
        fighters[lost_keys[-1]] = 3
        lost_keys = lost_keys[:-1]
    pairs = pairings(fighters)
    for p in pairs:
        before_1 = fighters[p[0]]
        before_2 = fighters[p[1]]
        fighters[p[0]] = fighters[p[0]] - random.randint(1,3)
        fighters[p[1]] = fighters[p[1]] - random.randint(1,3)    
        if(v >= 2):
            print(p[0], ':', before_1, '-->', fighters[p[0]])
            print(p[1], ':', before_2, '-->',  fighters[p[1]])
    return clean(fighters, lost_keys)

In [5]:
# Initial setup
def simulate(NUM_F = 10, v=1):
    fighters = {}
    lost_fighters = []
    START_HP = 12
    for i in range(NUM_F):
        fighters[i] = START_HP

    # rounds
    total_fights = 0
    rounds = 0
    while (len(fighters.keys()) > 6):
        rounds += 1
        fights_in_round = int((len(fighters.keys()) + 1)/2)
        total_fights += fights_in_round
        fighters, lost_fighters = conduct_round(fighters, lost_fighters,v)
        if v >= 1:
            print("Round {}".format(rounds))
            print("{} fights in round, {} fighters out, {} fighters left.".format(fights_in_round, len(lost_fighters), len(fighters.keys())))
            print("Total fights {}, time {}".format(total_fights, total_fights*2 + 5*rounds))
            print("{} total points left")
        if v >= 2:
            print(fighters)
            print(lost_fighters)
    return rounds, total_fights

In [6]:
import numpy as np

rounds = []
fights = []

for i in range (100):
    r, f = simulate(70,0)
    #print(r,f)
    rounds.append(r)
    fights.append(f)
print('Rounds: ', np.min(rounds), np.mean(rounds), np.max(rounds))
print('fights: ', np.min(fights), np.mean(fights), np.max(fights))

Rounds:  7 7.82 8
fights:  210 221.27 232


In [7]:
def tournament_time(rounds, fights, grounds):
    return (5*rounds + fights*3/grounds)/60

In [8]:
tournament_time(8, 230, 2)

6.416666666666667

# Swiss pairings

In [9]:
class Fighter:
    
    def __init__(self, name, hp=12):
        self.name = name
        self.hp = hp
        self.enemies = {}#{self.name:9999}
    
    def fight(self, other, hp_lost):
        if other.name in self.enemies.keys():
            self.enemies[other.name] += 1
        else:
            self.enemies[other.name] = 1
        self.hp -= hp_lost
        
    def played(self, other):
        if other.name in self.enemies.keys():
            return self.enemies[other.name]
        else:
            return 0

    def normalize_played(self, others):
        if len(self.enemies.keys()) < len(others):
            return
        m = 99
        for o in others:
            if o.name not in self.enemies.keys():
                return
            else:
                m = min(m, self.enemies[o.name])
        if m == 0:
            return
        # If we reached here it means that all others played at least 1 time
        # Then we substract 1 from all of them to nullify at least one of them
        print(self.name + " played with all at least {} times".format(m))
        for o in others:
            self.enemies[o.name] -= m
        
    def __repr__(self):
        return (self.name + ' ' + str(self.hp))

In [10]:
def fight(f1, f2, result):
    f1.fight(f2, result[0])
    f2.fight(f1, result[1])

def playerStandings(players):
    """Returns a list of the players and their win records, sorted by score.

    The first entry in the list should be the player in first place, or a player
    tied for first place if there is currently a tie.

    Args:
        None
    Returns:
      A list of tuples, each of which contains (id, name, wins, losses, draws, matches, score):
        (0)id: the player's unique id (assigned by the database)
        (1)name: the player's full name (as registered)
        (6)score: the score of this player
    """
    def hp(fighter):
        return fighter.hp
    
    return sorted(players, key=hp)

def alreadyPlayed(player1, player2):
    """Have these two players already played in this tour?

    Args:
        player1: player's unique id#
        player2: player's unique id#
        tour_id: tour's unique id#
    Returns:
        A boolean indicating if this would be a rematch
    """
    return player1.played(player2) > 0 or player2.played(player1) > 0

def normalize_all_played(fighters):
    for f in fighters:
        f.normalize_played([other for other in fighters if other.name != f.name])

def swissPairings(players):
    """Returns a list of pairs of players for the next round of a match in this tour.

    Assuming that there are an even number of players registered, each player
    appears exactly once in the pairings.  Each player is paired with another
    player with an equal or nearly-equal win record, that is, a player adjacent
    to him or her in the standings.  Rematches are not allowed, so all pairings are new.

    Returns:
      A list of tuples, each of which contains (id1, name1, id2, name2)
        id1: the first player's unique id
        name1: the first player's name
        id2: the second player's unique id
        name2: the second player's name
    """
    standings = playerStandings(players)
    pairings = []

    for i in range(0, len(standings), 2):
        for j in range(i+1, len(standings)):
            if not alreadyPlayed(standings[i], standings[j]):
                # good pair found, swap into place then break the inner loop
                while j > i + 1:
                    standings[j-1], standings[j] = standings[j], standings[j-1]
                    j -= 1
                break
        pairings.append((standings[i], standings[i+1]))
    return pairings

def remove(fighters, outs, v):
    new_outs = []
    m = 3
    for f in fighters:
        if f.hp <= 0:
            new_outs.append(f)
        else:
            m = min(m, f.hp)
            
    if len(new_outs)%2 != 0:
        lucky = random.choice(new_outs)
        if v >= 1:
            print ('Lucky {}'.format(lucky.name))
        lucky.hp = m
        new_outs.remove(lucky)
        
    for f in new_outs:
        fighters.remove(f)
        
    outs += new_outs

In [11]:
def conduct_round_swiss(fighters, outs, v):
    pairs = swissPairings(fighters)
    #print(pairs)
    for p in pairs:
        fight(p[0], p[1], (random.randint(1,3), random.randint(1,3)))
    normalize_all_played(fighters)
    remove(fighters, outs, v)

In [12]:
fighters = [Fighter(str(i)) for i in range(10)]

In [13]:
outs = []
conduct_round_swiss(fighters, outs)
fighters

TypeError: conduct_round_swiss() missing 1 required positional argument: 'v'

In [14]:
outs

[]

In [15]:
def simulate_swiss(NUM_F = 10, v=1):
    fighters = [Fighter(str(i)) for i in range(NUM_F)]
    outs = []

    # rounds
    total_fights = 0
    rounds = 0
    while (len(fighters) > 6):
        rounds += 1
        fights_in_round = int(len(fighters)/2)
        total_fights += fights_in_round
        conduct_round_swiss(fighters, outs, v)
        if v >= 1:
            print("Round {}".format(rounds))
            print("{} fights in round, {} fighters out, {} fighters left.".format(fights_in_round, len(outs), len(fighters)))
            print("Total fights {}".format(total_fights))
           # print("{} total points left")
        if v >= 2:
            print(fighters)
            print(outs)
    return rounds, total_fights

In [28]:
rounds = []
fights = []
for i in range (100):
    r, f = simulate_swiss(26,0)
    #print(r,f)
    rounds.append(r)
    fights.append(f)
print('Rounds: ', np.min(rounds), np.mean(rounds), np.max(rounds))
print('fights: ', np.min(fights), np.mean(fights), np.max(fights))
print(tournament_time(np.max(rounds), np.max(fights), 2))

Rounds:  6 6.98 8
fights:  71 81.1 88
2.86666666667


# Google sheets

In [2]:
import httplib2
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials as SAC
CREDENTIALS_FILE = './google_token.json'

credentials = SAC.from_json_keyfile_name(CREDENTIALS_FILE, ['https://www.googleapis.com/auth/spreadsheets',
                                                              'https://www.googleapis.com/auth/drive'])
httpAuth = credentials.authorize(httplib2.Http())
service = apiclient.discovery.build('sheets', 'v4', http=httpAuth)


In [1]:
spreadsheet = service.spreadsheets().create(body = {
    'properties': {'title': 'Mws-sheets', 'locale': 'ru_RU'},
    'sheets': [{'properties': {'sheetType': 'GRID',
                               'sheetId': 0,
                               'title': 'Round_1',
                               'gridProperties': {'rowCount': 23, 'columnCount': 15}}}]
}).execute()


In [2]:
spreadsheet

{'properties': {'autoRecalc': 'ON_CHANGE',
  'defaultFormat': {'backgroundColor': {'blue': 1, 'green': 1, 'red': 1},
   'padding': {'bottom': 2, 'left': 3, 'right': 3, 'top': 2},
   'textFormat': {'bold': False,
    'fontFamily': 'arial,sans,sans-serif',
    'fontSize': 10,
    'foregroundColor': {},
    'italic': False,
    'strikethrough': False,
    'underline': False},
   'verticalAlignment': 'BOTTOM',
   'wrapStrategy': 'OVERFLOW_CELL'},
  'locale': 'ru_RU',
  'timeZone': 'Etc/GMT',
  'title': 'Mws-sheets'},
 'sheets': [{'properties': {'gridProperties': {'columnCount': 15,
     'rowCount': 23},
    'index': 0,
    'sheetId': 0,
    'sheetType': 'GRID',
    'title': 'Round_1'}}],
 'spreadsheetId': '1fRVq3Rdenq4K5YEGeRSYiSSanJLJbCOX9NcCo6PNyQA',
 'spreadsheetUrl': 'https://docs.google.com/spreadsheets/d/1fRVq3Rdenq4K5YEGeRSYiSSanJLJbCOX9NcCo6PNyQA/edit'}

In [5]:
# Share for manual updates

driveService = apiclient.discovery.build('drive', 'v3', http = httpAuth)
shareRes = driveService.permissions().create(
    fileId = spreadsheet['spreadsheetId'],
    body = {'type': 'user', 'role': 'writer', 'emailAddress': 'alexey.trekin@gmail.com'},
    fields = 'id'
).execute()

In [ ]:
{
  "requests": [
    {
      "addSheet": {
        "properties": {
          "title": "Deposits",
          "gridProperties": {
            "rowCount": 20,
            "columnCount": 12
          },
          "tabColor": {
            "red": 1.0,
            "green": 0.3,
            "blue": 0.4
          }
        }
      }
    }
  ]
}

In [7]:
shareRes

{'id': '08110577062868256794i'}

In [12]:
# set column width
request = [] 
for col in [2,3,4,5,7,9,10,11,12]:
    request.append({
          "updateDimensionProperties": {
          "range": {
          "sheetId": 0,
          "dimension": "COLUMNS",  # COLUMNS - потому что столбец
          "startIndex": col,         # Столбцы нумеруются с нуля
          "endIndex": col+1            # startIndex берётся включительно, endIndex - НЕ включительно,
                                   # т.е. размер будет применён к столбцам в диапазоне [0,1), т.е. только к столбцу A
          },
          "properties": {
          "pixelSize": 35     # размер в пикселях
          },
          "fields": "pixelSize"  # нужно задать только pixelSize и не трогать другие параметры столбца
          }
    })

results = service.spreadsheets().batchUpdate(spreadsheetId = spreadsheet['spreadsheetId'], 
                                             body = {"requests": request}).execute()

In [3]:
spreadsheetId = '1P4cP-V8FWYa7Jwu1IEEWSQ1dmMG9vu6El-x54xspifQ'

In [8]:
# merge some cells
request = [
{'mergeCells': {'range': {'sheetId': 0,
                          'startRowIndex': 0,
                          'endRowIndex': 1,
                          'startColumnIndex': 1,
                          'endColumnIndex': 7},
                'mergeType': 'MERGE_ALL'}},
{'mergeCells': {'range': {'sheetId': 0,
                          'startRowIndex': 0,
                          'endRowIndex': 1,
                          'startColumnIndex': 8,
                          'endColumnIndex': 14},
                'mergeType': 'MERGE_ALL'}}]

results = service.spreadsheets().batchUpdate(spreadsheetId = spreadsheetId, 
                                             body = {"requests": request}).execute()

## Add sheet

In [5]:
request = [{'addSheet':
            {'properties': {
              "sheetId": 1,
              "title": 'TEST_SHEET',
              "index": 1 ,
            "gridProperties": {
                "rowCount": 15,
                "columnCount": 23
          },
            }
              }
           }]
results = service.spreadsheets().batchUpdate(spreadsheetId = spreadsheetId, 
                                             body = {"requests": request}).execute()

In [9]:
# Enter static data
data_request  = {
 "valueInputOption": "USER_ENTERED",
    "data": [
        {"range": "Round_1!B1:B1",
         "majorDimension": "ROWS",     # сначала заполнять ряды, затем столбцы (т.е. самые внутренние списки в values - это ряды)
         "values": [["1 ристалище"]]},

        {"range": "Round_1!I1:I1",
         "majorDimension": "COLUMNS",  # сначала заполнять столбцы, затем ряды (т.е. самые внутренние списки в values - это столбцы)
         "values": [["2 ристалище"]]},
        
        {"range": "Round_1!B2:N2",
         "majorDimension": "ROWS",     # сначала заполнять ряды, затем столбцы (т.е. самые внутренние списки в values - это ряды)
         "values": [["Фамилия", "HP", "Бой", "Бой","HP","Фамилия", "", "Фамилия", "HP", "Бой", "Бой","HP","Фамилия" ]]},
    ]}

results = service.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheetID, 
                                             body = data_request).execute()

In [10]:
def get_position(round_number, area, pair_num):
    # 1-based area
    if area == 1:
        columns = ('B', 'G')
    elif area == 2:
        columns = ('I', 'N')
    else: 
        raise ValueError("Only 0 and 1 are valid areas")
    sheet = 'Round_' + str(round_number)
    row = pair_num + 2 # heading
    return '{sheet}!{begin}{row}:{end}{row}'.format(sheet=sheet, row=row, begin = columns[0], end = columns[1])

def get_data(red_fighter, blue_fighter):
    return [red_fighter.name, red_fighter.hp, '', '', blue_fighter.hp, blue_fighter.name]
    
def write_pair(spreadsheetId, round_number, area, pair_num, red_fighter, blue_fighter):
    position = get_position(round_number, area, pair_num)
    pair_data = get_data(red_fighter, blue_fighter)
    data_request  = {
     "valueInputOption": "USER_ENTERED",
     "data": [
        {"range": position,
         "majorDimension": "ROWS",     # сначала заполнять ряды, затем столбцы (т.е. самые внутренние списки в values - это ряды)
         "values": [pair_data]}
    ]}
    results = service.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheetId, 
                                             body = data_request).execute()

In [11]:
get_position(1, 1, 1)

'Round_1!B3:G3'

In [15]:
from fighter import Fighter

blue = Fighter("Loki", 10)
red = Fighter("Thor", 12)

write_pair(spreadsheetId, 1, 2, 3, red, blue)

In [67]:
# Color
sheetId = 0
requests = [
        {'repeatCell': {'range': {'sheetId': sheetId,
                          'startRowIndex': 1,
                          'endRowIndex': 23,
                          'startColumnIndex': 1,
                          'endColumnIndex': 4},
                'cell': {"userEnteredFormat": {
                                       "backgroundColor": {
                                           "red": 1,
                                           "green": 0.6,
                                           "blue": 0.6,
                                           "alpha": 0.6
                                       }}},
                'fields': 'userEnteredFormat'}},
    
        {'repeatCell': {'range': {'sheetId': sheetId,
                          'startRowIndex': 1,
                          'endRowIndex': 23,
                          'startColumnIndex': 8,
                          'endColumnIndex': 11},
                'cell': {"userEnteredFormat": {
                                       "backgroundColor": {
                                           "red": 1,
                                           "green": 0.6,
                                           "blue": 0.6,
                                           "alpha": 0.6
                                       }}},
                'fields': 'userEnteredFormat'}},
    
    
        {'repeatCell': {'range': {'sheetId': sheetId,
                          'startRowIndex': 1,
                          'endRowIndex': 23,
                          'startColumnIndex': 4,
                          'endColumnIndex': 7},
                'cell': {"userEnteredFormat": {
                                       "backgroundColor": {
                                           "red": 0.6,
                                           "green": 0.6,
                                           "blue": 1,
                                           "alpha": 0.6
                                       }}},
                'fields': 'userEnteredFormat'}},
    
    
        {'repeatCell': {'range': {'sheetId': sheetId,
                          'startRowIndex': 1,
                          'endRowIndex': 23,
                          'startColumnIndex': 11,
                          'endColumnIndex': 14},
                'cell': {"userEnteredFormat": {
                                       "backgroundColor": {
                                           "red": 0.6,
                                           "green": 0.6,
                                           "blue": 1,
                                           "alpha": 0.6
                                       }}},
                'fields': 'userEnteredFormat'}},
    
            ]

body = {'requests': requests}

response = service.spreadsheets().batchUpdate(spreadsheetId=spreadsheetId, body=body).execute()

# Read data back from the sheet

In [52]:
import pprint
pp = pprint.PrettyPrinter()

request = service.spreadsheets().values().get(spreadsheetId=spreadsheetId, range='I3:N23')
response = request.execute()
pp.pprint(response)

{'majorDimension': 'ROWS',
 'range': 'Round_1!I3:N23',
 'values': [['Thor', '12', '', '', '10', 'Loki'],
            ['Thor', '12', '', '', '10', 'Loki'],
            ['Thor', '12', '', '', '10', 'Loki'],
            ['d'],
            [],
            ['d'],
            [],
            [],
            [],
            [],
            [],
            [],
            ['', '', '', '', '', 'd']]}


In [53]:
len(response['values'])

13

In [25]:
[0,1,2][::-1]

[2, 1, 0]

In [2]:
import io
with io.open('./tests/test_fighters', encoding='utf-8') as src:
    fighters = [s.split(',')[0] for s in src.readlines()]
print(fighters)

['Сурт', 'Якорь', 'Сайресс ', 'Kerulen', 'Rebrov', 'Poluga ', 'Sandra  ', 'Dika ', 'Gwen', 'Burger']


In [4]:
with io.open('/home/trekin/Data/test/out_fighters', 'w', encoding='utf-8') as dst:
    dst.write(fighters[0])

# Round pairings

In [18]:
from pairings.round_pairings import RoundPairings

In [92]:
def dumbRoundPairings(fighters):
    pairings = []
    f_num = len(fighters)
    for f1 in range(f_num):
        for f2 in range(f1):
            pairings.append((fighters[f1], fighters[f2]))
    return pairings

def subround_consequent(fighters):
    # Return list of the consequent pairings, like (i, i+1)
    pairings = []
    i=0
    
    # Odd pairs (1,2),(3,4),... (7,1)
    while i < len(fighters):
        pairings.append((fighters[i], fighters[(i+1)%len(fighters)]))
        i+= 2
    # Even pairs (2,3), (4,5), ... (8,1)
    i = 1
    while i < len(fighters):
        if i == len(fighters) - 1:
            # Special case - when the number of fighters is even, 
            # we want the last pair to be inserted in the previous position 
            # to avoid consequent fights for the first fighter
            pairings.insert( -1, (fighters[i], fighters[(i+1)%len(fighters)]))
        else:
            pairings.append((fighters[i], fighters[(i+1)%len(fighters)]))
        i += 2
    # Special case - when the number of fighters is even, 

    return pairings

def subround_half_shift(fighters):
    pairings = []
    if len(fighters)%2 != 0:
        return []
    for i in range(len(fighters)//2):
        pairings.append((fighters[i], fighters[i + len(fighters)//2]))
    return pairings

def subround(fighters, shift):
    L = len(fighters)

    if shift == 1:
        # If the pairs are consequent, general algorithm would make 2 fights in a row for one fighter
        # So it is a special case
        return subround_consequent(fighters)
    if L%2 == 0 and L//2 == shift:
        return subround_half_shift(fighters)
    pairings = []
    for i in range(L):
        pairings.append((fighters[i], fighters[(i+shift)%L]))
    return pairings

def RoundPairings_(fighters):
    pairings = []
    for shift in range(1, len(fighters)//2 + 1):
        pairings += subround(fighters, shift)
    return pairings

In [69]:
def test_pairings(fighters, pairings):
    fights = {}
    for i in range(len(pairings)-1):
        if pairings[i][0] not in fights.keys():
            fights[pairings[i][0]] = 1
        else:
            fights[pairings[i][0]] += 1
            
        if pairings[i][1] not in fights.keys():
            fights[pairings[i][1]] = 1
        else:
            fights[pairings[i][1]] += 1

            
        if (pairings[i][0] == pairings[i][1]):
            print("Error: pair {} has the same fighter in both positions".format(i))
        if pairings[i][0] == pairings[i+1][0] or pairings[i][0]== pairings[i+1][1] or \
           pairings[i][1] == pairings[i+1][0] or pairings[i][1]== pairings[i+1][1]:
            print("Error: two fights in a row in position {}".format(i))
        for j in range(i+1, len(pairings)):
            if pairings[i] == pairings[j] or pairings[i][0]== pairings[j][1] and pairings[j][0]== pairings[i][1]:
                print("Error: pairs {} and {} are the same".format(i,j))
    
    # Add the last fight
    if pairings[-1][0] not in fights.keys():
        fights[pairings[-1][0]] = 1
    else:
        fights[pairings[-1][0]] += 1
    
    if pairings[-1][1] not in fights.keys():
        fights[pairings[-1][1]] = 1
    else:
        fights[pairings[-1][1]] += 1
            
    if sorted(fighters) != sorted(list(fights.keys())):
        print("Fighters list not covered")
        print(fights.keys())

    for f, f_num in fights.items():
        if f_num != len(fighters) - 1:
            print("fighter {} has {} fights, not {}".format(f, f_num, len(fighters) - 1))

In [93]:
fighters = [1,2,3]
pairings = RoundPairings_(fighters)
print(pairings)
test_pairings(fighters, pairings)
# This error is inevitable here

[(1, 2), (3, 1), (2, 3)]
Error: two fights in a row in position 0
Error: two fights in a row in position 1


In [94]:
fighters = [1,2,3,4]
pairings = RoundPairings_(fighters)
print(pairings)
test_pairings(fighters, pairings)
# These errors are also OK

[(1, 2), (3, 4), (4, 1), (2, 3), (1, 3), (2, 4)]
Error: two fights in a row in position 1
Error: two fights in a row in position 3


In [107]:
for i in range (5, 50):
    fighters = [j for j in range(1, i+1)]
    pairings = RoundPairings_(fighters)
    test_pairings(fighters, pairings)
# OK!

In [111]:
# Special cases
print(RoundPairings_([]))
print(RoundPairings_([1]))
print(RoundPairings_([1,2]))

[]
[]
[(2, 1), (1, 2)]


In [142]:
randint(a=-4, b=0)

-1

# Test swiss pairings - statistical

In [3]:
from random import randint

def generate_sample_results(num_fights):
    return [(randint(-3,0), randint(-3,0)) for i in range(num_fights)]

generate_sample_results(10)

[(-2, -3),
 (-2, -2),
 (0, -1),
 (0, -3),
 (0, -3),
 (-1, -1),
 (-3, -2),
 (0, -3),
 (0, -2),
 (-1, -3)]

In [5]:
from pairings.swiss_pairings import swiss_pairings, swiss_pairings_old


In [6]:
class TestSwissPairings:

    # Test normal behavior
    def test_num_fights_in_a_round(self):
        for fighters in generate_data_all():
            pairings = swiss_pairings(fighters)
            assert len(pairings) == len(fighters)/2

    def test_one_fight_for_fighter_in_a_round(self):
        for fighters in generate_data_all():
            pairings = swiss_pairings(fighters)
            fights = {}
            for i in range(len(pairings)):
                if pairings[i][0] not in fights.keys():
                    fights[pairings[i][0].name] = 1
                else:
                    fights[pairings[i][0].name] += 1

                if pairings[i][1] not in fights.keys():
                    fights[pairings[i][1].name] = 1
                else:
                    fights[pairings[i][1].name] += 1

            # Fighters list is the same
            assert sorted([f.name for f in fighters]) == sorted(list(fights.keys()))

            # Every one gets one fight
            for f, f_num in fights.items():
                assert f_num == 1

    def test_no_self_fight(self):
        for fighters in generate_data_all():
            pairings = swiss_pairings(fighters)
            for p in pairings:
                assert p[0] != p[1]

    def test_no_repeated_fight(self):
        for fighters in generate_data_all():
            pairings = swiss_pairings(fighters)
            for p in pairings:
                assert p[1].name not in p[0].enemies
                assert p[0].name not in p[1].enemies

    # test special cases
    def test_error_on_odd_number(self):
        fighters = [Fighter(name=str(i + 1)) for i in range(11)]
        with pytest.raises(ValueError):
            swiss_pairings(fighters)

In [363]:
class test_api:
    def __init__(self, cap):
        self.all_fights = []
        self.cap = cap
        
    def write(self, pairs, round_num):
        self.pairs = [(p[0].name, p[1].name) for p in pairs]
        
    def read(self, round_num):
        results = []
        for pair in self.pairs:
            res = [-self.cap, randint(-self.cap,0)]
            if(randint(0,1) == 0):
                res = [res[1], res[0]]
            results.append(((pair[0], res[0]),(pair[1], res[1])))
        self.all_fights.append(results)
        return results

In [360]:
from tournament.tournament import Tournament, Fighter

In [493]:
def conduct_tournament(pairing_function = swiss_pairings_old, fighters_num=16, hp = 20, cap = 6, depth=5):
    fighters = [Fighter(hp=hp, name=str(i)) for i in range(fighters_num)]
    tour = Tournament(fightCap=cap, fighters=fighters, pairing_function=pairing_function)
    api = test_api(cap=cap)

    r=0
    while tour.remove() is None:
        tour.make_pairs()

        # test no repeated fights - the only thing we really need here
        for p in tour.pairings:
            if p[1].name in p[0].enemies or p[0].name in p[1].enemies:
                print(f'FAIL in round {r+1} at {p}')
                #for r in api.all_fights:
                #    print(r)
                #print(tour.pairings)
                return False
        
        tour.write_pairs(api, r)
        tour.read_results(api, r)
        #print(tour.fighters)
        r+= 1
    print("SUCCESS!")#, api.all_fights)
    return True

In [495]:
conduct_tournament(pairing_function = swiss_pairings_old, fighters_num=8, hp=20, cap=6)

We have the finalists:
[2, 9, 5, 6, 6, 4]
SUCCESS!


True

# New variant

In [510]:
import numpy as np

def already_played(player1: Fighter, player2: Fighter) -> bool:
    """Have these two players already played in this tour?

        A boolean indicating if this would be a rematch
    """
    return player1.played(player2) > 0 or player2.played(player1) > 0

def hp(fighter: Fighter) -> int:
    # Function to sort fighters
    return fighter.hp

def get_one_pair(fighters):
    first = fighters[0]
    pairs = [(first, second) for second in fighters[1:] if not already_played(first, second)]
    return pairs

class Candidate:
    def __init__(self, pairs, fighters):
        self.pairs = pairs
        self.remaining = [f for f in fighters if not any(f in p for p in pairs)]

        diff = [abs(p[0].hp - p[1].hp) for p in pairs]
        if not diff:
            self.max_diff = 0
            self.tot_diff = 0
        else:
            self.max_diff = np.max(diff)
            self.tot_diff = np.sum(diff)

    def add_pair(self, pair):
        return Candidate(self.pairs + [pair], self.remaining)
    
    def __repr__(self):
        return ';'.join(str(self.pairs))

def swiss_pairings(fighters, max_diff=-1, candidates_to_keep=5):
    """Returns a list of pairs of players for the next round of a match in this tour.

    Assuming that there are an even number of players registered, each player
    appears exactly once in the pairings.  Each player is paired with another
    player with an equal or nearly-equal HP, that is, a player adjacent
    to him or her in the standings.  Rematches are not allowed, so all pairings are new
    (excluding situations when all pairs have matched, see Fighter.normalize_played )

    Returns: a list of tuples of fighters
    """
    if len(fighters) % 2 != 0 or len(fighters) == 0:
        raise ValueError("Number of fighters is {}, does not suit for pairing".format(len(fighters)))

    standings = sorted(fighters, key=hp, reverse=True)
    # Dynamic programming method with width-search and cutoff of the bad variants
    # We start from pairing all the players with the first one
    # Then we add the next pair to each first pair until all are paired
    # Keeping these requirements:
    # 1. Without the same pairing twice
    # 2. Minimizing the maximum point difference between matches
    # 3. Between the variants of 2. we minimize the total point difference.
    # First version will be pretty straightforward and will take much time and memory

    candidates = [Candidate([], standings)]
    for i in range(len(standings)//2):
        new_candidates = []
        for c in candidates:
            first = c.remaining[0]
            for second in c.remaining[1:]:
                if not already_played(first,second) and \
                        (abs(first.hp - second.hp) <= max_diff or max_diff < 0):
                    new_candidates.append(c.add_pair((first, second)))
        if len(new_candidates) == 0:
            print(f'{len(candidates)} Candidates')
            for c in candidates:
                print(c.pairs)
                print(c.remaining)
                first = c.remaining[0]
                for second in c.remaining[1:]:
                    print (first, second, already_played(first,second))
                
        candidates = sorted(new_candidates,
                            key=lambda candidate: candidate.max_diff)[0:min(candidates_to_keep, len(new_candidates))]
    
    # candidates = sorted(candidates, key=lambda candidate: candidate.max_diff*len(standings)*10 + candidate.tot_diff)
    
    if len(candidates) == 0:
        print('Error, returning default pairs')
        return [(standings[i*2], standings[i*2+1]) for i in range(len(standings)//2)]
    #print(candidates[0].max_diff)
    return candidates[0].pairs

def swiss_pairings_10(fighters, max_diff=-1, candidates_to_keep=10):
    """Returns a list of pairs of players for the next round of a match in this tour.

    Assuming that there are an even number of players registered, each player
    appears exactly once in the pairings.  Each player is paired with another
    player with an equal or nearly-equal HP, that is, a player adjacent
    to him or her in the standings.  Rematches are not allowed, so all pairings are new
    (excluding situations when all pairs have matched, see Fighter.normalize_played )

    Returns: a list of tuples of fighters
    """
    if len(fighters) % 2 != 0 or len(fighters) == 0:
        raise ValueError("Number of fighters is {}, does not suit for pairing".format(len(fighters)))

    standings = sorted(fighters, key=hp, reverse=True)
    # Dynamic programming method with width-search and cutoff of the bad variants
    # We start from pairing all the players with the first one
    # Then we add the next pair to each first pair until all are paired
    # Keeping these requirements:
    # 1. Without the same pairing twice
    # 2. Minimizing the maximum point difference between matches
    # 3. Between the variants of 2. we minimize the total point difference.
    # First version will be pretty straightforward and will take much time and memory

    candidates = [Candidate([], standings)]
    for i in range(len(standings)//2):
        new_candidates = []
        for c in candidates:
            first = c.remaining[0]
            for second in c.remaining[1:]:
                if not already_played(first,second) and \
                        (abs(first.hp - second.hp) <= max_diff or max_diff < 0):
                    new_candidates.append(c.add_pair((first, second)))
        if len(new_candidates) == 0:
            print(f'{len(candidates)} Candidates')
            for c in candidates:
                print(c.pairs)
                print(c.remaining)
                first = c.remaining[0]
                for second in c.remaining[1:]:
                    print (first, second, already_played(first,second))
                
        candidates = sorted(new_candidates,
                            key=lambda candidate: candidate.max_diff)[0:min(candidates_to_keep, len(new_candidates))]
    
    # candidates = sorted(candidates, key=lambda candidate: candidate.max_diff*len(standings)*10 + candidate.tot_diff)
    
    if len(candidates) == 0:
        print('Error, returning default pairs')
        return [(standings[i*2], standings[i*2+1]) for i in range(len(standings)//2)]
    #print(candidates[0].max_diff)
    return candidates[0].pairs


In [508]:
attempts = 100    
fails = 0
for run in range(attempts):
    if not conduct_tournament(pairing_function = swiss_pairings_old, fighters_num=26, hp=20, cap=5):
        fails += 1
print(f'{fails} of {attempts} failed') 

FAIL in round 3 at (23, 10, 24, 10)
FAIL in round 4 at (19, 5, 22, 5)
FAIL in round 6 at (14, 1, 15, 1)
Lucky one: 5, -2
Lucky one: 20, -4
FAIL in round 7 at (12, 1, 20, 1)
Lucky one: 24, 0
Lucky one: 11, 0
We have the finalists:
[4, 10, 6, 2, 7, 2, 13, 1, 17, 1, 25, 4]
SUCCESS!
FAIL in round 3 at (21, 10, 24, 10)
Lucky one: 5, 0
We have the finalists:
[6, 3, 13, 5, 18, 3, 19, 3, 20, 2]
SUCCESS!
Lucky one: 16, 0
Lucky one: 14, -2
We need to setup an additional round to choose finalists. Ready finalists are:
[7, 3, 24, 1]
Candidates for additional round:
[2, -1, 6, -1, 14, -4, 17, 0, 18, 0, 20, -2, 21, -3, 23, -2]
SUCCESS!
Lucky one: 6, 0
Lucky one: 2, -1
We have the finalists:
[1, 3, 7, 1, 19, 5]
SUCCESS!
FAIL in round 2 at (24, 15, 25, 15)
FAIL in round 4 at (21, 5, 22, 5)
FAIL in round 5 at (24, 2, 14, 1)
FAIL in round 2 at (24, 15, 25, 15)
FAIL in round 3 at (22, 10, 23, 10)
FAIL in round 5 at (18, 1, 24, 1)
Lucky one: 11, -4
FAIL in round 7 at (12, 1, 13, 1)
Lucky one: 20, 0
Lucky 

In [509]:
attempts = 1000
fails = 0
for run in range(attempts):
    if not conduct_tournament(pairing_function = swiss_pairings, fighters_num=26, hp=20, cap=5):
        fails += 1
print(f'{fails} of {attempts} failed') 

We have the finalists:
[0, 1, 1, 1, 16, 1, 22, 3, 25, 13]
SUCCESS!
Lucky one: 16, 0
We have the finalists:
[4, 2, 10, 1, 16, 1, 17, 4, 18, 1, 23, 5]
SUCCESS!
We need to setup an additional round to choose finalists. Ready finalists are:
[1, 2, 15, 1]
Candidates for additional round:
[0, -4, 2, -4, 5, -3, 12, -4, 14, -2, 18, 0, 21, -1, 25, 0]
SUCCESS!
Lucky one: 10, 0
We have the finalists:
[8, 4, 9, 2, 15, 1, 17, 3]
SUCCESS!
Lucky one: 9, -4
We need to setup an additional round to choose finalists. Ready finalists are:
[]
Candidates for additional round:
[2, -4, 5, -1, 9, -4, 13, -4, 15, -1, 16, -4, 19, -4, 25, -3]
SUCCESS!
We have the finalists:
[1, 3, 8, 1, 12, 1]
SUCCESS!
Lucky one: 13, 0
We have the finalists:
[6, 1, 10, 6, 11, 5]
SUCCESS!
Lucky one: 8, 0
We need to setup an additional round to choose finalists. Ready finalists are:
[18, 4]
Candidates for additional round:
[0, 0, 2, -2, 5, -1, 7, -4, 8, -3, 9, 0, 10, -2, 23, -4, 24, -2]
SUCCESS!
5 Candidates
[(2, 12, 23, 12), (20, 

[2, 3, 8, 2, 11, 1, 15, 6, 16, 2]
SUCCESS!
Lucky one: 2, 0
We have the finalists:
[0, 1, 1, 5, 5, 1, 7, 2, 8, 1, 16, 15]
SUCCESS!
Lucky one: 12, 0
Lucky one: 8, -2
We have the finalists:
[3, 4, 6, 2, 16, 1]
SUCCESS!
Lucky one: 2, 0
We have the finalists:
[6, 1, 17, 1, 25, 7]
SUCCESS!
We have the finalists:
[8, 1, 15, 8, 16, 4, 19, 1, 23, 2, 25, 5]
SUCCESS!
Lucky one: 25, 0
Lucky one: 3, -4
We have the finalists:
[0, 1, 4, 1, 16, 5, 18, 7]
SUCCESS!
Lucky one: 17, 0
Lucky one: 25, -4
We have the finalists:
[9, 1, 11, 6, 16, 3, 20, 1, 24, 2]
SUCCESS!
Lucky one: 20, 0
Lucky one: 19, -3
We have the finalists:
[6, 1, 14, 6, 16, 1, 25, 4]
SUCCESS!
Lucky one: 20, 0
Lucky one: 9, -1
We need to setup an additional round to choose finalists. Ready finalists are:
[]
Candidates for additional round:
[2, 0, 6, -4, 7, -1, 10, -2, 14, -2, 17, 0, 18, 0, 23, 0]
SUCCESS!
Lucky one: 11, 0
We need to setup an additional round to choose finalists. Ready finalists are:
[13, 1, 18, 8]
Candidates for additiona

[10, 4, 20, 4]
Candidates for additional round:
[0, -2, 4, -3, 5, -2, 9, -4, 13, -3, 15, -4, 18, -3, 24, -4]
SUCCESS!
We have the finalists:
[8, 6, 16, 7, 17, 3, 24, 3, 25, 4]
SUCCESS!
Lucky one: 25, -3
Lucky one: 14, -1
We need to setup an additional round to choose finalists. Ready finalists are:
[13, 9, 18, 2]
Candidates for additional round:
[3, -1, 8, -4, 12, -1, 14, -2, 17, -4, 23, -2]
SUCCESS!
Lucky one: 5, -2
We have the finalists:
[1, 8, 6, 1, 8, 6, 12, 2, 16, 3, 24, 2]
SUCCESS!
Lucky one: 15, -1
Lucky one: 5, -1
We have the finalists:
[6, 8, 10, 1, 20, 2]
SUCCESS!
Lucky one: 14, 0
Lucky one: 21, -3
We have the finalists:
[6, 3, 19, 4, 25, 1]
SUCCESS!
We have the finalists:
[2, 3, 3, 1, 5, 1, 10, 1, 16, 3, 23, 5]
SUCCESS!
Lucky one: 13, -3
Lucky one: 4, -1
We have the finalists:
[7, 5, 8, 5, 11, 3, 24, 2]
SUCCESS!
Lucky one: 12, 0
Lucky one: 1, -2
We have the finalists:
[3, 3, 4, 5, 11, 10, 15, 1, 18, 3]
SUCCESS!
We have the finalists:
[6, 8, 10, 1, 12, 3, 15, 3, 20, 5, 24, 5]

Lucky one: 1, 0
5 Candidates
[(4, 13, 13, 12), (11, 9, 14, 9), (6, 8, 0, 7), (16, 7, 25, 7), (17, 7, 19, 7), (3, 6, 12, 6), (20, 6, 9, 5), (10, 5, 18, 5), (5, 4, 7, 3), (24, 4, 2, 3), (21, 3, 15, 2)]
[1, 1, 22, 1]
1, 1 22, 1 True
[(4, 13, 13, 12), (11, 9, 14, 9), (6, 8, 0, 7), (16, 7, 25, 7), (17, 7, 19, 7), (3, 6, 12, 6), (20, 6, 9, 5), (10, 5, 18, 5), (5, 4, 7, 3), (24, 4, 21, 3), (2, 3, 15, 2)]
[1, 1, 22, 1]
1, 1 22, 1 True
[(4, 13, 13, 12), (11, 9, 14, 9), (6, 8, 0, 7), (16, 7, 25, 7), (17, 7, 19, 7), (3, 6, 12, 6), (20, 6, 9, 5), (10, 5, 18, 5), (5, 4, 21, 3), (24, 4, 2, 3), (7, 3, 15, 2)]
[1, 1, 22, 1]
1, 1 22, 1 True
[(4, 13, 13, 12), (11, 9, 14, 9), (6, 8, 0, 7), (16, 7, 25, 7), (17, 7, 19, 7), (3, 6, 12, 6), (20, 6, 9, 5), (10, 5, 18, 5), (5, 4, 21, 3), (24, 4, 7, 3), (2, 3, 15, 2)]
[1, 1, 22, 1]
1, 1 22, 1 True
[(4, 13, 13, 12), (11, 9, 14, 9), (6, 8, 0, 7), (16, 7, 25, 7), (17, 7, 19, 7), (3, 6, 12, 6), (20, 6, 9, 5), (10, 5, 5, 4), (18, 5, 24, 4), (2, 3, 7, 3), (21, 3, 15, 

We have the finalists:
[0, 3, 3, 1, 9, 1, 14, 1]
SUCCESS!
Lucky one: 7, 0
Lucky one: 7, -4
We have the finalists:
[17, 6, 18, 3, 22, 2]
SUCCESS!
Lucky one: 14, 0
We have the finalists:
[2, 1, 8, 7, 10, 9, 16, 4, 17, 2, 24, 1]
SUCCESS!
Lucky one: 15, 0
We have the finalists:
[4, 4, 5, 2, 10, 1, 18, 4]
SUCCESS!
We have the finalists:
[4, 2, 6, 5, 17, 1, 19, 5, 21, 5]
SUCCESS!
Lucky one: 15, 0
We have the finalists:
[1, 1, 10, 1, 14, 5, 15, 1, 25, 1]
SUCCESS!
Lucky one: 21, 0
We have the finalists:
[0, 6, 2, 5, 16, 1, 20, 2, 23, 12]
SUCCESS!
Lucky one: 12, 0
Lucky one: 25, -1
Lucky one: 0, -1
We need to setup an additional round to choose finalists. Ready finalists are:
[2, 1, 4, 5]
Candidates for additional round:
[0, -4, 3, -2, 13, -3, 20, -1, 21, -3, 24, -2]
SUCCESS!
We have the finalists:
[10, 7, 14, 4, 15, 2, 22, 1]
SUCCESS!
Lucky one: 9, 0
Lucky one: 9, -4
We have the finalists:
[4, 2, 6, 2, 12, 1, 16, 3, 23, 2]
SUCCESS!
Lucky one: 8, 0
We have the finalists:
[2, 4, 9, 2, 15, 8, 17,

We have the finalists:
[1, 3, 11, 7, 18, 1, 20, 2, 21, 3, 25, 8]
SUCCESS!
Lucky one: 12, 0
We have the finalists:
[4, 4, 9, 6, 25, 3]
SUCCESS!
We have the finalists:
[3, 6, 8, 5, 9, 1, 10, 3, 14, 4, 20, 2]
SUCCESS!
Lucky one: 18, 0
Lucky one: 17, -1
We need to setup an additional round to choose finalists. Ready finalists are:
[13, 1, 23, 7]
Candidates for additional round:
[3, 0, 6, -4, 12, -3, 16, -1, 22, -3, 25, 0]
SUCCESS!
Lucky one: 14, 0
We have the finalists:
[4, 2, 5, 10, 7, 2, 10, 2, 15, 1]
SUCCESS!
Lucky one: 24, 0
Lucky one: 2, -1
We have the finalists:
[1, 4, 3, 3, 10, 5]
SUCCESS!
We have the finalists:
[8, 4, 13, 2, 20, 1, 24, 6]
SUCCESS!
Lucky one: 5, -2
We need to setup an additional round to choose finalists. Ready finalists are:
[6, 1, 9, 1]
Candidates for additional round:
[1, -2, 10, -4, 12, -4, 16, -2, 17, -1, 19, -2]
SUCCESS!
Lucky one: 18, -2
We have the finalists:
[4, 6, 9, 4, 11, 3, 13, 4, 19, 3, 23, 3]
SUCCESS!
Lucky one: 21, -1
We have the finalists:
[0, 2, 3,

Candidates for additional round:
[0, -4, 3, -1, 4, -4, 14, -3, 20, -1, 22, -3]
SUCCESS!
Lucky one: 24, 0
We have the finalists:
[5, 2, 7, 6, 22, 4]
SUCCESS!
Lucky one: 22, 0
We have the finalists:
[0, 1, 7, 3, 9, 3, 20, 6, 25, 1]
SUCCESS!
Lucky one: 23, -3
We have the finalists:
[1, 6, 8, 3, 14, 2, 15, 3, 16, 1, 22, 3]
SUCCESS!
We have the finalists:
[9, 2, 12, 7, 14, 2, 16, 1, 23, 3]
SUCCESS!
We have the finalists:
[3, 5, 7, 6, 15, 7, 23, 4]
SUCCESS!
Lucky one: 19, 0
Lucky one: 3, -2
We have the finalists:
[4, 4, 18, 4, 21, 7]
SUCCESS!
Lucky one: 7, -2
We have the finalists:
[12, 4, 14, 3, 16, 4, 19, 5, 22, 1]
SUCCESS!
We have the finalists:
[3, 7, 7, 7, 11, 1, 14, 2]
SUCCESS!
Lucky one: 8, 0
We have the finalists:
[3, 3, 6, 2, 13, 7, 16, 3, 24, 1, 25, 3]
SUCCESS!
Lucky one: 18, 0
Lucky one: 22, -1
We have the finalists:
[3, 1, 4, 7, 8, 2, 16, 4, 17, 4, 23, 11]
SUCCESS!
Lucky one: 9, 0
Lucky one: 7, -1
We have the finalists:
[0, 1, 2, 3, 12, 6, 14, 1, 19, 3]
SUCCESS!
Lucky one: 0, -4


We need to setup an additional round to choose finalists. Ready finalists are:
[9, 8, 17, 1]
Candidates for additional round:
[0, -4, 1, -1, 4, -4, 12, -4, 13, -2, 23, -4]
SUCCESS!
Lucky one: 2, 0
We have the finalists:
[0, 2, 11, 1, 15, 3, 20, 2, 21, 3, 22, 3]
SUCCESS!
Lucky one: 7, 0
Lucky one: 10, -2
We have the finalists:
[0, 9, 2, 4, 6, 2, 9, 5]
SUCCESS!
We have the finalists:
[3, 2, 8, 2, 10, 5, 13, 5, 19, 3, 23, 5]
SUCCESS!
Lucky one: 24, 0
Lucky one: 11, -4
Lucky one: 7, -2
We need to setup an additional round to choose finalists. Ready finalists are:
[8, 2, 14, 1]
Candidates for additional round:
[5, -4, 7, 0, 10, 0, 16, -3, 17, -4, 19, 0]
SUCCESS!
Lucky one: 12, 0
We have the finalists:
[3, 2, 8, 4, 11, 5, 16, 3, 18, 1, 25, 2]
SUCCESS!
Lucky one: 11, 0
Lucky one: 13, -4
Lucky one: 13, -2
We have the finalists:
[9, 2, 12, 4, 15, 6]
SUCCESS!
Lucky one: 12, 0
Lucky one: 16, -2
Lucky one: 10, 0
We need to setup an additional round to choose finalists. Ready finalists are:
[7, 2, 

[5, 9, 13, 1, 24, 6]
SUCCESS!
Lucky one: 14, 0
Lucky one: 14, -4
We have the finalists:
[9, 4, 11, 2, 12, 2, 16, 3, 22, 3]
SUCCESS!
Lucky one: 10, 0
We have the finalists:
[5, 1, 9, 6, 22, 2]
SUCCESS!
Lucky one: 7, -2
Lucky one: 17, -1
We have the finalists:
[12, 4, 14, 6, 16, 2]
SUCCESS!
Lucky one: 9, 0
We have the finalists:
[12, 9, 15, 2, 22, 3]
SUCCESS!
Lucky one: 20, 0
We have the finalists:
[2, 7, 7, 2, 9, 2, 22, 6, 23, 2, 24, 1]
SUCCESS!
Lucky one: 20, -1
We have the finalists:
[1, 5, 4, 3, 7, 2, 9, 4, 14, 3, 15, 2]
SUCCESS!
Lucky one: 2, -4
We have the finalists:
[7, 1, 11, 3, 19, 7, 20, 5, 23, 2, 25, 4]
SUCCESS!
Lucky one: 23, -3
We have the finalists:
[2, 4, 5, 2, 7, 2, 11, 1]
SUCCESS!
Lucky one: 13, 0
We have the finalists:
[4, 2, 9, 1, 14, 2, 18, 5, 23, 2]
SUCCESS!
Lucky one: 22, 0
Lucky one: 16, -1
We have the finalists:
[9, 2, 10, 1, 18, 3]
SUCCESS!
Lucky one: 2, -4
We have the finalists:
[0, 3, 4, 2, 19, 2, 20, 6]
SUCCESS!
We have the finalists:
[0, 5, 2, 4, 5, 1, 6, 3, 

We need to setup an additional round to choose finalists. Ready finalists are:
[2, 1, 17, 2]
Candidates for additional round:
[0, -4, 7, -4, 8, -4, 19, 0, 23, -3, 24, -2]
SUCCESS!
Lucky one: 14, 0
Lucky one: 7, -1
Lucky one: 23, -1
We need to setup an additional round to choose finalists. Ready finalists are:
[12, 5]
Candidates for additional round:
[3, -2, 5, -1, 8, -3, 9, -3, 17, -3, 18, 0, 21, -2, 22, -4, 23, -4]
SUCCESS!
Lucky one: 13, 0
Lucky one: 13, -4
Lucky one: 12, -2
We need to setup an additional round to choose finalists. Ready finalists are:
[14, 1, 17, 4]
Candidates for additional round:
[7, 0, 10, 0, 12, -3, 21, -4, 22, -2, 23, -1]
SUCCESS!
We have the finalists:
[0, 6, 4, 5, 6, 2, 7, 1, 9, 4, 21, 1]
SUCCESS!
Lucky one: 0, 0
We have the finalists:
[1, 2, 8, 8, 16, 2, 25, 1]
SUCCESS!
Lucky one: 17, -2
Lucky one: 2, 0
We have the finalists:
[14, 4, 18, 3, 20, 9]
SUCCESS!
Lucky one: 22, 0
Lucky one: 19, -3
Lucky one: 24, -2
We need to setup an additional round to choose fin

In [511]:
attempts = 1000
fails = 0
for run in range(attempts):
    if not conduct_tournament(pairing_function = swiss_pairings_10, fighters_num=26, hp=20, cap=5):
        fails += 1
print(f'{fails} of {attempts} failed') 

Lucky one: 15, -1
We have the finalists:
[7, 5, 9, 1, 14, 10, 16, 3, 17, 5, 23, 3]
SUCCESS!
Lucky one: 24, 0
Lucky one: 20, 0
Lucky one: 20, -3
We need to setup an additional round to choose finalists. Ready finalists are:
[10, 5]
Candidates for additional round:
[0, -3, 2, -2, 6, -4, 8, 0, 9, -1, 14, 0, 16, 0, 20, -4, 22, -1]
SUCCESS!
Lucky one: 14, 0
We have the finalists:
[2, 4, 5, 8, 13, 1, 23, 4, 25, 3]
SUCCESS!
Lucky one: 20, 0
We have the finalists:
[10, 6, 17, 1, 19, 6, 24, 4]
SUCCESS!
We need to setup an additional round to choose finalists. Ready finalists are:
[6, 4, 24, 1]
Candidates for additional round:
[4, -3, 9, 0, 12, -3, 14, -2, 21, -4, 25, -2]
SUCCESS!
Lucky one: 17, 0
Lucky one: 22, -2
Lucky one: 19, -3
We have the finalists:
[0, 7, 7, 2, 16, 1, 23, 1]
SUCCESS!
Lucky one: 1, -3
Lucky one: 1, -3
We need to setup an additional round to choose finalists. Ready finalists are:
[5, 1, 23, 1]
Candidates for additional round:
[1, 0, 3, -4, 12, -2, 13, -4, 20, -2, 25, 0]
SUC

[3, 2, 9, 2, 17, 3, 21, 3]
SUCCESS!
Lucky one: 20, 0
We have the finalists:
[6, 5, 8, 6, 12, 2, 17, 1, 18, 1, 23, 2]
SUCCESS!
Lucky one: 5, 0
We have the finalists:
[0, 2, 6, 1, 9, 8, 21, 1, 23, 1]
SUCCESS!
We have the finalists:
[4, 4, 8, 2, 10, 3, 15, 5, 17, 3, 24, 2]
SUCCESS!
Lucky one: 25, 0
We have the finalists:
[1, 12, 7, 5, 10, 3, 13, 4, 14, 3, 15, 2]
SUCCESS!
We have the finalists:
[3, 5, 16, 1, 17, 1, 23, 5, 24, 5, 25, 1]
SUCCESS!
Lucky one: 0, 0
We have the finalists:
[5, 4, 10, 1, 12, 1, 15, 4, 18, 8]
SUCCESS!
Lucky one: 0, 0
Lucky one: 8, 0
We have the finalists:
[4, 3, 5, 7, 11, 1, 12, 6, 20, 3, 24, 1]
SUCCESS!
Lucky one: 25, 0
Lucky one: 21, -1
We have the finalists:
[11, 3, 12, 5, 18, 7, 24, 7]
SUCCESS!
Lucky one: 2, 0
Lucky one: 24, -2
We have the finalists:
[0, 4, 5, 2, 13, 3, 17, 5, 20, 6]
SUCCESS!
We have the finalists:
[8, 11, 11, 3, 15, 2, 24, 5]
SUCCESS!
We have the finalists:
[6, 3, 9, 4, 15, 1, 21, 3]
SUCCESS!
Lucky one: 11, 0
Lucky one: 18, -1
We have the fina

[1, 1, 5, 3, 7, 2, 8, 6, 24, 7, 25, 4]
SUCCESS!
We have the finalists:
[2, 3, 8, 5, 18, 14, 23, 2, 24, 3, 25, 1]
SUCCESS!
Lucky one: 13, 0
Lucky one: 13, -4
We need to setup an additional round to choose finalists. Ready finalists are:
[]
Candidates for additional round:
[2, -4, 3, -2, 5, -3, 8, -4, 12, -2, 16, -2, 17, -3, 21, -1]
SUCCESS!
We have the finalists:
[1, 1, 2, 13, 4, 3, 5, 11, 22, 2, 24, 1]
SUCCESS!
Lucky one: 22, -3
Lucky one: 10, -3
We need to setup an additional round to choose finalists. Ready finalists are:
[5, 6, 11, 4]
Candidates for additional round:
[0, -1, 7, -2, 9, -2, 10, -4, 17, -4, 21, -2]
SUCCESS!
Lucky one: 16, 0
Lucky one: 25, -3
We have the finalists:
[0, 2, 4, 2, 10, 8, 12, 3, 20, 2]
SUCCESS!
We have the finalists:
[2, 3, 5, 3, 7, 1, 9, 3, 15, 2, 20, 1]
SUCCESS!
Lucky one: 16, 0
Lucky one: 16, -4
We have the finalists:
[4, 1, 11, 5, 22, 1, 25, 1]
SUCCESS!
Lucky one: 8, 0
We have the finalists:
[2, 4, 7, 6, 16, 3, 23, 1]
SUCCESS!
We have the finalists:
[0,

[18, 1, 19, 1, 20, 10]
SUCCESS!
Lucky one: 12, 0
Lucky one: 6, -4
We need to setup an additional round to choose finalists. Ready finalists are:
[1, 1, 24, 4]
Candidates for additional round:
[6, -4, 8, -2, 10, -3, 11, 0, 13, 0, 14, -1, 17, -2, 19, -4]
SUCCESS!
Lucky one: 7, -1
We have the finalists:
[8, 7, 15, 2, 16, 1]
SUCCESS!
We have the finalists:
[1, 6, 4, 2, 5, 1, 9, 1, 18, 1, 22, 8]
SUCCESS!
Lucky one: 24, 0
Lucky one: 14, 0
We have the finalists:
[2, 4, 6, 2, 8, 2, 12, 3, 13, 3, 16, 3]
SUCCESS!
Lucky one: 21, 0
Lucky one: 21, -4
We have the finalists:
[1, 1, 2, 6, 4, 2, 6, 1, 15, 1, 22, 7]
SUCCESS!
Lucky one: 22, -1
We have the finalists:
[1, 8, 7, 4, 11, 1, 16, 4]
SUCCESS!
We have the finalists:
[0, 3, 2, 5, 13, 6, 20, 2, 22, 7]
SUCCESS!
Lucky one: 7, 0
Lucky one: 3, 0
Lucky one: 12, -3
We have the finalists:
[4, 4, 8, 3, 10, 1]
SUCCESS!
We have the finalists:
[4, 6, 6, 1, 16, 9, 21, 2]
SUCCESS!
Lucky one: 2, 0
Lucky one: 11, -2
Lucky one: 15, -3
We need to setup an additiona

[4, 4, 7, 4, 19, 3, 21, 1]
SUCCESS!
Lucky one: 19, -4
We have the finalists:
[3, 1, 5, 1, 10, 3, 18, 2, 25, 1]
SUCCESS!
We have the finalists:
[2, 3, 10, 1, 15, 6, 17, 1, 23, 5]
SUCCESS!
Lucky one: 24, -3
We have the finalists:
[7, 1, 8, 2, 12, 7, 16, 1, 18, 8, 24, 1]
SUCCESS!
Lucky one: 17, -2
Lucky one: 9, -1
We have the finalists:
[12, 1, 15, 5, 25, 1]
SUCCESS!
We have the finalists:
[5, 2, 8, 1, 13, 1]
SUCCESS!
Lucky one: 14, 0
Lucky one: 6, -1
We need to setup an additional round to choose finalists. Ready finalists are:
[3, 1, 24, 2]
Candidates for additional round:
[1, -4, 4, 0, 5, -4, 6, -4, 10, -1, 13, -2, 14, -4, 17, -3]
SUCCESS!
Lucky one: 25, 0
We have the finalists:
[0, 5, 4, 4, 6, 2, 7, 1, 21, 4]
SUCCESS!
Lucky one: 3, 0
We have the finalists:
[2, 5, 5, 4, 9, 5, 15, 2, 18, 1, 20, 9]
SUCCESS!
Lucky one: 1, 0
Lucky one: 5, -4
We have the finalists:
[9, 9, 19, 1, 24, 3]
SUCCESS!
Lucky one: 5, 0
We have the finalists:
[1, 2, 9, 2, 11, 4, 18, 3, 19, 5, 21, 1]
SUCCESS!
Lucky on

We need to setup an additional round to choose finalists. Ready finalists are:
[17, 7]
Candidates for additional round:
[0, 0, 2, 0, 5, -1, 7, 0, 11, -4, 16, 0, 21, -1]
SUCCESS!
Lucky one: 2, 0
We have the finalists:
[1, 1, 5, 2, 8, 1, 10, 4, 14, 6, 17, 1]
SUCCESS!
Lucky one: 5, 0
We have the finalists:
[4, 1, 7, 1, 13, 1, 16, 5, 17, 4, 23, 1]
SUCCESS!
Lucky one: 4, -1
We need to setup an additional round to choose finalists. Ready finalists are:
[11, 2, 14, 2]
Candidates for additional round:
[5, -4, 6, -3, 9, -2, 15, -2, 18, 0, 20, 0, 22, -2, 24, 0]
SUCCESS!
Lucky one: 7, 0
Lucky one: 23, -1
Lucky one: 23, -3
We have the finalists:
[5, 3, 23, 1, 25, 1]
SUCCESS!
Lucky one: 9, 0
We need to setup an additional round to choose finalists. Ready finalists are:
[17, 4]
Candidates for additional round:
[3, -1, 8, -2, 10, -4, 11, -3, 15, -4, 19, 0, 25, 0]
SUCCESS!
Lucky one: 11, -1
We have the finalists:
[2, 2, 8, 3, 21, 1, 22, 2]
SUCCESS!
Lucky one: 16, -3
Lucky one: 16, -4
We need to setup 

Lucky one: 12, -2
We have the finalists:
[8, 4, 9, 6, 14, 4, 15, 5, 16, 5, 25, 5]
SUCCESS!
Lucky one: 7, 0
Lucky one: 9, -4
We have the finalists:
[5, 2, 16, 1, 20, 2]
SUCCESS!
Lucky one: 24, 0
We have the finalists:
[10, 3, 13, 1, 15, 2, 22, 1]
SUCCESS!
Lucky one: 21, -1
We have the finalists:
[1, 2, 4, 2, 5, 8, 9, 2, 13, 1, 23, 5]
SUCCESS!
We have the finalists:
[11, 4, 14, 4, 16, 4]
SUCCESS!
Lucky one: 2, 0
Lucky one: 19, -1
We need to setup an additional round to choose finalists. Ready finalists are:
[]
Candidates for additional round:
[1, -1, 7, -1, 8, -1, 15, -3, 18, -4, 19, -2, 22, -2, 24, -4]
SUCCESS!
Lucky one: 23, 0
We have the finalists:
[2, 2, 4, 4, 9, 2, 11, 5, 19, 4]
SUCCESS!
Lucky one: 4, -3
We have the finalists:
[3, 4, 9, 1, 10, 5, 16, 1, 21, 4]
SUCCESS!
We have the finalists:
[12, 2, 14, 1, 17, 5, 19, 1, 21, 3, 23, 2]
SUCCESS!
Lucky one: 11, 0
We need to setup an additional round to choose finalists. Ready finalists are:
[4, 3, 22, 2]
Candidates for additional round:

[4, 1, 7, 4, 12, 3]
SUCCESS!
We have the finalists:
[10, 3, 16, 4, 24, 1]
SUCCESS!
Lucky one: 5, 0
Lucky one: 22, -4
We need to setup an additional round to choose finalists. Ready finalists are:
[2, 1, 13, 2]
Candidates for additional round:
[0, -3, 4, 0, 8, -4, 12, 0, 22, -1, 23, 0]
SUCCESS!
Lucky one: 14, -1
We have the finalists:
[1, 2, 4, 4, 6, 1, 11, 2, 23, 2, 25, 1]
SUCCESS!
Lucky one: 11, -4
We have the finalists:
[7, 2, 11, 1, 24, 7]
SUCCESS!
Lucky one: 24, 0
Lucky one: 18, -3
Lucky one: 23, 0
We need to setup an additional round to choose finalists. Ready finalists are:
[]
Candidates for additional round:
[1, -3, 7, -3, 12, -2, 14, 0, 15, -4, 19, -2, 20, -4, 23, -3]
SUCCESS!
Lucky one: 22, 0
Lucky one: 21, -1
We need to setup an additional round to choose finalists. Ready finalists are:
[10, 7, 19, 1]
Candidates for additional round:
[0, -4, 4, -1, 9, -3, 17, -2, 21, -1, 23, -4]
SUCCESS!
Lucky one: 5, 0
We have the finalists:
[2, 3, 4, 3, 14, 5, 17, 1, 20, 4, 21, 1]
SUCCESS!


SUCCESS!
Lucky one: 10, 0
We have the finalists:
[4, 3, 5, 4, 7, 5, 8, 8, 24, 6, 25, 1]
SUCCESS!
Lucky one: 0, 0
Lucky one: 15, 0
Lucky one: 21, -1
We have the finalists:
[3, 1, 17, 14, 21, 1]
SUCCESS!
We have the finalists:
[0, 1, 5, 3, 9, 4, 23, 8]
SUCCESS!
Lucky one: 1, 0
We have the finalists:
[0, 4, 2, 1, 9, 3, 11, 2, 13, 10, 24, 4]
SUCCESS!
Lucky one: 16, 0
Lucky one: 20, 0
We have the finalists:
[2, 3, 6, 2, 11, 7, 21, 1, 23, 2, 24, 2]
SUCCESS!
Lucky one: 7, 0
Lucky one: 11, 0
We have the finalists:
[0, 2, 15, 2, 16, 1]
SUCCESS!
Lucky one: 10, -3
We have the finalists:
[2, 1, 5, 1, 14, 1, 19, 8]
SUCCESS!
Lucky one: 16, 0
We have the finalists:
[3, 2, 15, 2, 19, 1]
SUCCESS!
Lucky one: 5, -2
We have the finalists:
[6, 2, 10, 4, 11, 2, 17, 2, 22, 1, 25, 2]
SUCCESS!
We need to setup an additional round to choose finalists. Ready finalists are:
[]
Candidates for additional round:
[0, 0, 3, -4, 4, -2, 7, -1, 8, -4, 10, -1, 13, -1, 25, -3]
SUCCESS!
Lucky one: 20, 0
Lucky one: 11, 0
We 

In [1]:
from TM.tournament.fighter import Fighter, Division
from TM.tournament.fight import Fight
import json

input_data = json.loads(open('./tests/fighter_json').read())

f1 = Fighter(**input_data)
f2 = Fighter(fighter_id='K', first_name='Kevin', last_name='Brown')


In [6]:
f1.country = 'UK'
f1.club = 'MI6'
f1.city = 'London'
f1.rating = 10
f1.__dict__

{'city': 'London',
 'club': 'MI6',
 'country': 'UK',
 'division': <Division.tc: 'tc'>,
 'doubles': 0,
 'enemies': {},
 'fighter_id': '007',
 'fights': [],
 'first_name': 'James',
 'global_rating': 9000,
 'last_name': 'Bond',
 'rating': 10,
 'warnings': 0}

In [7]:
fight = Fight(f2, f1)
fight.__dict__

{'current_round': 0,
 'current_time': 0,
 'doubles': 0,
 'fighter_1': Brown Kevin, 0,
 'fighter_2': Bond James, 10,
 'round_score_1': 0,
 'round_score_2': 0,
 'score_1': 0,
 'score_2': 0,
 'status': <FightStatus.planned: 'planned'>,
 'warnings_1': 0,
 'warnings_2': 0}

In [8]:
fight.to_dict()

{'current_round': 0,
 'current_time': 0,
 'doubles': 0,
 'fighter_1': 'K',
 'fighter_2': '007',
 'round_score_1': 0,
 'round_score_2': 0,
 'score_1': 0,
 'score_2': 0,
 'status': 'planned',
 'warnings_1': 0,
 'warnings_2': 0}

In [12]:
fighters = [f1,f2]
standings = sorted(fighters, key=lambda f: f.rating, reverse=False)

In [14]:
standings

[Brown Kevin, 0, Bond James, 10]

In [1]:
from TM.tournament.fighter import Fighter
from TM.tournament.fight import Fight, Round, FightStatus

In [2]:
foo = Fighter('007', 'James', 'Bond')
print(foo.__dict__)
print(foo.to_dict)

{'fighter_id': '007', 'first_name': 'James', 'last_name': 'Bond', 'club': '', 'city': '', 'country': '', 'rating': 0, 'global_rating': 0, 'division': <Division.base: 'base'>, 'warnings': 0, 'doubles': 0, 'fights': [], 'enemies': {}}
{'fighter_id': '007', 'first_name': 'James', 'last_name': 'Bond', 'club': '', 'city': '', 'country': '', 'rating': 0, 'global_rating': 0, 'division': 'base', 'warnings': 0, 'doubles': 0, 'fights': [], 'enemies': {}}


In [3]:
bar = Fight(foo, foo)
print(bar.__dict__)
print(bar.to_dict)

{'fighter_1': Bond James, 0, 'fighter_2': Bond James, 0, 'status': <FightStatus.planned: 'planned'>, 'rounds_num': 1, 'current_round': 0, 'rounds': [<TM.tournament.fight.Round object at 0x7ffb446c83c8>]}
{'fighter_1': {'fighter_id': '007', 'first_name': 'James', 'last_name': 'Bond', 'club': '', 'city': '', 'country': '', 'rating': 0, 'global_rating': 0, 'division': 'base', 'warnings': 0, 'doubles': 0, 'fights': [], 'enemies': {}}, 'fighter_2': {'fighter_id': '007', 'first_name': 'James', 'last_name': 'Bond', 'club': '', 'city': '', 'country': '', 'rating': 0, 'global_rating': 0, 'division': 'base', 'warnings': 0, 'doubles': 0, 'fights': [], 'enemies': {}}, 'status': 'planned', 'rounds_num': 1, 'current_round': 0, 'rounds': [{'score_1': 0, 'score_2': 0, 'status': 'planned', 'doubles': 0, 'warnings_1': 0, 'warnings_2': 0, 'current_time': 0, 'result_1': None, 'result_2': None}], 'doubles': 0, 'warnings_1': 0, 'warnings_2': 0}


In [4]:
foo2= Fighter('K', 'Kevin', 'Smith')

In [5]:
bar = Fight(foo,foo2)
bar

In [6]:
bar.to_dict

{'current_round': 0,
 'doubles': 0,
 'fighter_1': {'city': '',
  'club': '',
  'country': '',
  'division': 'base',
  'doubles': 0,
  'enemies': {},
  'fighter_id': '007',
  'fights': [],
  'first_name': 'James',
  'global_rating': 0,
  'last_name': 'Bond',
  'rating': 0,
  'warnings': 0},
 'fighter_2': {'city': '',
  'club': '',
  'country': '',
  'division': 'base',
  'doubles': 0,
  'enemies': {},
  'fighter_id': 'K',
  'fights': [],
  'first_name': 'Kevin',
  'global_rating': 0,
  'last_name': 'Smith',
  'rating': 0,
  'warnings': 0},
 'rounds': [{'current_time': 0,
   'doubles': 0,
   'result_1': None,
   'result_2': None,
   'score_1': 0,
   'score_2': 0,
   'status': 'planned',
   'warnings_1': 0,
   'warnings_2': 0}],
 'rounds_num': 1,
 'status': 'planned',
 'warnings_1': 0,
 'warnings_2': 0}

In [8]:
bar.fighter_1.division.value

'base'

In [12]:
class Pairings():
    def __init__(self):
        self.A = 'a'
    def __call__(self):
        print(self.A, 'AAA')
        

In [14]:
Pairings()()

a AAA
